<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/POS%2B_RNN_%2BLSTM__task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP group project/data')

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
import pandas as pd

In [ ]:
# df = pd.read_csv('final_task1_data.csv')

In [ ]:
# df = df[['text','label']]


In [4]:
import nltk
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Download the required NLTK corpora
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')


In [ ]:
# Load the data frame
df = pd.read_csv('final_task1_data.csv')

In [ ]:
# Define a function for POS tagging
def pos_tagging(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Perform POS tagging on the words
    pos_tags = nltk.pos_tag(words)
    # Return the POS tagged text
    return pos_tags

In [ ]:
# Apply the POS tagging function to the 'text' column
df['pos_tags'] = df['Generation'].apply(pos_tagging)


In [ ]:
# Encode the labels using the LabelEncoder class
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

In [ ]:
df = df[['text','label']]

In [ ]:
import numpy as np
import pandas as pd
import nltk
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SimpleRNN, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# Download the required NLTK corpora
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

In [ ]:
# Define the maximum length of the input sequences
max_length = 100

# Define the number of training epochs
epochs = 10

# Define the number of units in the LSTM layer
lstm_units = 128

# Define the number of units in the RNN layer
rnn_units = 64

# Define the size of the embedding layer
embedding_size = 128

# Define the batch size for training
batch_size = 32


In [ ]:
# Define a function for POS tagging
def pos_tagging(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Perform POS tagging on the words
    pos_tags = nltk.pos_tag(words)
    # Return the list of POS tags
    return [tag for word, tag in pos_tags]

# Apply the POS tagging function to the 'text' column
df['pos_tags'] = df['text'].apply(pos_tagging)

In [ ]:
# Encode the POS tags and create the input and output sequences
input_sequences = []
output_sequences = []
pos_tags = set()

for tags in df['pos_tags']:
    for tag in tags:
        pos_tags.add(tag)

In [ ]:
tag_encoder = LabelEncoder()
tag_encoder.fit(list(pos_tags))

In [ ]:
for tags in df['pos_tags']:
    input_sequence = []
    output_sequence = []
    for i in range(len(tags)):
        input_sequence.append(tags[i-1] if i > 0 else '')
        output_sequence.append(tags[i])
    input_sequences.append(input_sequence)
    output_sequences.append(output_sequence)


In [ ]:
# Convert the input and output sequences to integer sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_sequences)
tokenized_input_sequences = tokenizer.texts_to_sequences(input_sequences)
tokenized_output_sequences = tokenizer.texts_to_sequences(output_sequences)

In [ ]:
# Pad the input and output sequences to the same length
padded_input_sequences = pad_sequences(tokenized_input_sequences, maxlen=max_length, padding='post')
padded_output_sequences = pad_sequences(tokenized_output_sequences, maxlen=max_length, padding='post')


In [ ]:
# Encode the output sequences as one-hot vectors
one_hot_output_sequences = np.array([pd.get_dummies(tag_encoder.transform(seq)) for seq in padded_output_sequences])


In [ ]:
# Divide the data into training, validation, and test sets
train_input, test_input, train_output, test_output = train_test_split(padded_input_sequences, one_hot_output_sequences, test_size=0.2, random_state=42)
train_input, val_input, train_output, val_output = train_test_split(train_input, train_output, test_size=0.2, random_state=42)

In [ ]:
# Create the model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, embedding_size, input_length=max_length))
model.add(LSTM(lstm_units, return_sequences=True))
model.add(SimpleRNN(rnn_units, return_sequences=True))
model.add(Dense(len(pos_tags), activation='softmax'))

In [ ]:
# Compile the model
model

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_input, train_output, batch_size=batch_size, epochs=epochs, validation_data=(val_input, val_output))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_input, test_output, batch_size=batch_size)

# Predict the POS tags for the test set
y_pred = model.predict(test_input)




In [ ]:
# Convert the one-hot vectors to integer sequences
y_true = [tag_encoder.inverse_transform(np.argmax(seq, axis=1)) for seq in test_output]
y_pred = [tag_encoder.inverse_transform(np.argmax(seq, axis=1)) for seq in y_pred]

# Flatten the integer sequences
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)



In [ ]:
# Calculate the F1 score, accuracy, recall, and precision
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')

# Print the evaluation metrics
print('Test loss: {:.3f}'.format(test_loss))
print('Test accuracy: {:.3f}'.format(test_accuracy))
print('F1 score: {:.3f}'.format(f1))
print('Accuracy: {:.3f}'.format(accuracy))
print('Recall: {:.3f}'.format(recall))
print('Precision: {:.3f}'.format(precision))